<a href="https://colab.research.google.com/github/Adrianus-Bagas/Content-Based-Filtering-Recommender-System/blob/main/Content_Based_Filtering_Recommender_System_(Training).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Rekomendasi Film Indonesia

## 1. Import beberapa libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random
from numpy import nan

## 2. Memuat data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movie = pd.read_csv('/content/drive/MyDrive/dataset/indonesian_movies.csv')
movie

,title,year,description,genre,rating,users_rating,votes,languages,directors,actors,runtime
0,#FriendButMarried 2,2020,Ayudia (Mawar De Jongh) is not satisfied enoug...,Biography,13+,6.5,120,Indonesian,Rako Prijanto,"['Adipati Dolken', 'Mawar Eva de Jongh', 'Vonn...",100 min
1,4 Mantan,2020,"Sara, Airin, Rachel, and Amara were accidental...",Thriller,17+,6.4,8,Indonesian,Hanny Saputra,"['Ranty Maria', 'Jeff Smith', 'Melanie Berentz...",80 min
2,Aku Tahu Kapan Kamu Mati,2020,"After apparent death, Siena is able to see sig...",Horror,13+,5.4,17,Indonesian,Hadrah Daeng Ratu,"['Natasha Wilona', 'Ria Ricis', 'Al Ghazali', ...",92 min
3,Anak Garuda,2020,"Good Morning Indonesia, a school for poor orph...",Adventure,13+,9.1,27,Indonesian,Faozan Rizal,"['Tissa Biani Azzahra', 'Violla Georgie', 'Aji...",129 min
4,Dignitate,2020,Alfi (Al Ghazali) meets Alana (Caitlin Halderm...,Drama,17+,7.6,33,Indonesian,Fajar Nugros,"['Al Ghazali', 'Caitlin Halderman', 'Giorgino ...",109 min
...,...,...,...,...,...,...,...,...,...,...,...
1267,The Tiger from Tjampa,1953,"Set in the 1930s, and narrated like a ballad f...",Drama,NaN,6.4,30,Indonesian,D. Djajakusuma,"['Wahid Chan', 'Bambang Hermanto', 'R.D. Ismai...",97 min
1268,Enam Djam di Djogja,1951,Depicting the celebrated recapture of the town...,Drama,NaN,6.3,9,Indonesian,Usmar Ismail,"['R.D. Ismail', 'Del Juzar', 'Aedy Moward', 'A...",116 min
1269,Darah dan Doa,1950,It tells the story of an Indonesian revolution...,Drama,NaN,6.6,27,Indonesian,Usmar Ismail,"['Ella Bergen', 'Faridah', 'R.D. Ismail', 'Del...",150 min
1270,Resia Boroboedoer,1928,NaN,Adventure,NaN,7.0,8,Indonesian,NaN,"['Olive Young', nan, nan, nan, nan, nan, nan, ...",NaN


In [ ]:
movie=movie.dropna(subset=['description'])
movie=movie.dropna(subset=['actors'])

In [ ]:
movie

,title,year,description,genre,rating,users_rating,votes,languages,directors,actors,runtime
0,#FriendButMarried 2,2020,Ayudia (Mawar De Jongh) is not satisfied enoug...,Biography,13+,6.5,120,Indonesian,Rako Prijanto,"['Adipati Dolken', 'Mawar Eva de Jongh', 'Vonn...",100 min
1,4 Mantan,2020,"Sara, Airin, Rachel, and Amara were accidental...",Thriller,17+,6.4,8,Indonesian,Hanny Saputra,"['Ranty Maria', 'Jeff Smith', 'Melanie Berentz...",80 min
2,Aku Tahu Kapan Kamu Mati,2020,"After apparent death, Siena is able to see sig...",Horror,13+,5.4,17,Indonesian,Hadrah Daeng Ratu,"['Natasha Wilona', 'Ria Ricis', 'Al Ghazali', ...",92 min
3,Anak Garuda,2020,"Good Morning Indonesia, a school for poor orph...",Adventure,13+,9.1,27,Indonesian,Faozan Rizal,"['Tissa Biani Azzahra', 'Violla Georgie', 'Aji...",129 min
4,Dignitate,2020,Alfi (Al Ghazali) meets Alana (Caitlin Halderm...,Drama,17+,7.6,33,Indonesian,Fajar Nugros,"['Al Ghazali', 'Caitlin Halderman', 'Giorgino ...",109 min
...,...,...,...,...,...,...,...,...,...,...,...
1265,Tamu Agung,1955,An impending visit from a VIP causes chaos and...,Comedy,NaN,5.0,7,Indonesian,Usmar Ismail,"['Cassin Abbas', 'Nina Amora', 'M. Pandji Anom...",107 min
1266,Lewat Djam Malam,1954,A hero of revolution returns to civilian life ...,Drama,NaN,7.5,94,Indonesian,Usmar Ismail,"['A.N. Alcaff', 'Dhalia', 'Netty Herawati', 'B...",101 min
1267,The Tiger from Tjampa,1953,"Set in the 1930s, and narrated like a ballad f...",Drama,NaN,6.4,30,Indonesian,D. Djajakusuma,"['Wahid Chan', 'Bambang Hermanto', 'R.D. Ismai...",97 min
1268,Enam Djam di Djogja,1951,Depicting the celebrated recapture of the town...,Drama,NaN,6.3,9,Indonesian,Usmar Ismail,"['R.D. Ismail', 'Del Juzar', 'Aedy Moward', 'A...",116 min


In [ ]:
df_movie = movie.copy()
df_movie

,title,year,description,genre,rating,users_rating,votes,languages,directors,actors,runtime
0,#FriendButMarried 2,2020,Ayudia (Mawar De Jongh) is not satisfied enoug...,Biography,13+,6.5,120,Indonesian,Rako Prijanto,"['Adipati Dolken', 'Mawar Eva de Jongh', 'Vonn...",100 min
1,4 Mantan,2020,"Sara, Airin, Rachel, and Amara were accidental...",Thriller,17+,6.4,8,Indonesian,Hanny Saputra,"['Ranty Maria', 'Jeff Smith', 'Melanie Berentz...",80 min
2,Aku Tahu Kapan Kamu Mati,2020,"After apparent death, Siena is able to see sig...",Horror,13+,5.4,17,Indonesian,Hadrah Daeng Ratu,"['Natasha Wilona', 'Ria Ricis', 'Al Ghazali', ...",92 min
3,Anak Garuda,2020,"Good Morning Indonesia, a school for poor orph...",Adventure,13+,9.1,27,Indonesian,Faozan Rizal,"['Tissa Biani Azzahra', 'Violla Georgie', 'Aji...",129 min
4,Dignitate,2020,Alfi (Al Ghazali) meets Alana (Caitlin Halderm...,Drama,17+,7.6,33,Indonesian,Fajar Nugros,"['Al Ghazali', 'Caitlin Halderman', 'Giorgino ...",109 min
...,...,...,...,...,...,...,...,...,...,...,...
1265,Tamu Agung,1955,An impending visit from a VIP causes chaos and...,Comedy,NaN,5.0,7,Indonesian,Usmar Ismail,"['Cassin Abbas', 'Nina Amora', 'M. Pandji Anom...",107 min
1266,Lewat Djam Malam,1954,A hero of revolution returns to civilian life ...,Drama,NaN,7.5,94,Indonesian,Usmar Ismail,"['A.N. Alcaff', 'Dhalia', 'Netty Herawati', 'B...",101 min
1267,The Tiger from Tjampa,1953,"Set in the 1930s, and narrated like a ballad f...",Drama,NaN,6.4,30,Indonesian,D. Djajakusuma,"['Wahid Chan', 'Bambang Hermanto', 'R.D. Ismai...",97 min
1268,Enam Djam di Djogja,1951,Depicting the celebrated recapture of the town...,Drama,NaN,6.3,9,Indonesian,Usmar Ismail,"['R.D. Ismail', 'Del Juzar', 'Aedy Moward', 'A...",116 min


In [ ]:
df_movie["actors"] = df_movie['actors'].apply(lambda x: eval(x))

In [ ]:
df_movie

,title,year,description,genre,rating,users_rating,votes,languages,directors,actors,runtime
0,#FriendButMarried 2,2020,Ayudia (Mawar De Jongh) is not satisfied enoug...,Biography,13+,6.5,120,Indonesian,Rako Prijanto,"[Adipati Dolken, Mawar Eva de Jongh, Vonny Cor...",100 min
1,4 Mantan,2020,"Sara, Airin, Rachel, and Amara were accidental...",Thriller,17+,6.4,8,Indonesian,Hanny Saputra,"[Ranty Maria, Jeff Smith, Melanie Berentz, Den...",80 min
2,Aku Tahu Kapan Kamu Mati,2020,"After apparent death, Siena is able to see sig...",Horror,13+,5.4,17,Indonesian,Hadrah Daeng Ratu,"[Natasha Wilona, Ria Ricis, Al Ghazali, Fitria...",92 min
3,Anak Garuda,2020,"Good Morning Indonesia, a school for poor orph...",Adventure,13+,9.1,27,Indonesian,Faozan Rizal,"[Tissa Biani Azzahra, Violla Georgie, Ajil Dit...",129 min
4,Dignitate,2020,Alfi (Al Ghazali) meets Alana (Caitlin Halderm...,Drama,17+,7.6,33,Indonesian,Fajar Nugros,"[Al Ghazali, Caitlin Halderman, Giorgino Abrah...",109 min
...,...,...,...,...,...,...,...,...,...,...,...
1265,Tamu Agung,1955,An impending visit from a VIP causes chaos and...,Comedy,NaN,5.0,7,Indonesian,Usmar Ismail,"[Cassin Abbas, Nina Amora, M. Pandji Anom, Chi...",107 min
1266,Lewat Djam Malam,1954,A hero of revolution returns to civilian life ...,Drama,NaN,7.5,94,Indonesian,Usmar Ismail,"[A.N. Alcaff, Dhalia, Netty Herawati, Bambang ...",101 min
1267,The Tiger from Tjampa,1953,"Set in the 1930s, and narrated like a ballad f...",Drama,NaN,6.4,30,Indonesian,D. Djajakusuma,"[Wahid Chan, Bambang Hermanto, R.D. Ismail, Ma...",97 min
1268,Enam Djam di Djogja,1951,Depicting the celebrated recapture of the town...,Drama,NaN,6.3,9,Indonesian,Usmar Ismail,"[R.D. Ismail, Del Juzar, Aedy Moward, Agus Mul...",116 min


## 3. Text Preprocessing

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text

# Buat kolom tambahan untuk data description yang telah dibersihkan   
movie['desc_clean'] = movie['description'].apply(clean_text)

# 4. TFIDF dan Cosine Similarity

In [ ]:
movie.set_index('title', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movie['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.01312632, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01312632, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.01727536],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.04401851],
       [0.        , 0.        , 0.        , ..., 0.01727536, 0.04401851,
        1.        ]])

In [ ]:
cos_sim.shape

(840, 840)

In [ ]:
# Set index utama di kolom 'name'
indices = pd.Series(movie.index)
indices[:50]

0                       #FriendButMarried 2
1                                  4 Mantan
2                  Aku Tahu Kapan Kamu Mati
3                               Anak Garuda
4                                 Dignitate
5                           Guru-Guru Gokil
6     Hunter in the Blue Side of Manchester
7                                     Janin
8                                Mangkujiwo
9                                  Mariposa
10      May the Devil Take You: Chapter Two
11                                    Milea
12       Nanti Kita Cerita Tentang Hari Ini
13                                Nikah Yuk
14                                  Rasuk 2
15                      Si Doel the Movie 3
16                      Surat dari Kematian
17                          Temen Kondangan
18             Titus: Mystery of the Enygma
19                       Toko Barang Mantan
20                  #Malam Jumat: The Movie
21                               #MoveOnAja
22                          27 S

In [ ]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_hotel = []
    
    # Mengambil nama film berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 5 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:6].index)
    
    for i in top_10_indexes:
        recommended_hotel.append(list(movie.index)[i])
        
    return recommended_hotel

# 5. Implementasi

In [ ]:
print('Sistem : Hai pengguna Netfilm, kami ingin tahu film favoritmu. Boleh sebutkan 1 film favorit kamu ?')
inp = input()
df_film = df_movie[df_movie["title"]==inp]
rec = recommendations(inp)
print('Sistem : Jadi kamu suka genre',df_film.iloc[0]["genre"])
print('Sistem : Berikut 5 film rekomendasi kami yang sesuai dengan film favorit kamu : ')
print("==="*15)
for i in range(len(rec)):
  print(str(i+1)+". "+rec[i]+"\nGenre  : "+df_movie[df_movie["title"]==rec[i]].iloc[0]["genre"]+"\nTahun  : "
  +str(df_movie[df_movie["title"]==rec[i]].iloc[0]["year"])+"\nPemain : "
  +df_movie[df_movie["title"]==rec[i]].iloc[0]["actors"][0]+", "
  +df_movie[df_movie["title"]==rec[i]].iloc[0]["actors"][1]+", "
  +df_movie[df_movie["title"]==rec[i]].iloc[0]["actors"][2])
  print("==="*15)

Sistem : Hai pengguna Netfilm, kami ingin tahu film favoritmu. Boleh sebutkan 1 film favorit kamu ?
Dilan 1990
Sistem : Jadi kamu suka genre Drama
Sistem : Berikut 5 film rekomendasi kami yang sesuai dengan film favorit kamu : 
1. Milea
Genre  : Drama
Tahun  : 2020
Pemain : Iqbaal Dhiafakhri Ramadhan, Vanesha Prescilla, Ira Wibowo
2. Dilan 1991
Genre  : Drama
Tahun  : 2019
Pemain : Iqbaal Dhiafakhri Ramadhan, Vanesha Prescilla, Ira Wibowo
3. Milli & Nathan
Genre  : Drama
Tahun  : 2011
Pemain : Olivia Lubis Jensen, Christ Laurent, Sabai Morscheck
4. #FriendButMarried
Genre  : Biography
Tahun  : 2018
Pemain : Adipati Dolken, Vanesha Prescilla, Refal Hady
5. Stip & Pensil
Genre  : Comedy
Tahun  : 2017
Pemain : Ernest Prakasa, Tatjana Saphira, Indah Permatasari
